In [30]:
!pip install -U langchain langchain-community langchain-text-splitters langchain-huggingface huggingface_hub sentence-transformers faiss-cpu

  Using cached huggingface_hub-1.0.1-py3-none-any.whl.metadata (13 kB)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
!pip install transformers accelerate


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
!pip install -U langchain langchain-community huggingface_hub


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
# Core imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

# Hugging Face modules
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub

# LangChain core utilities
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os, glob, getpass


In [33]:
# Prompt for Hugging Face token securely
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your Hugging Face token: ")

# Optional: test authentication
from huggingface_hub import whoami
print("✅ Authenticated as:", whoami()["name"])


Enter your Hugging Face token:  ········


LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `hf auth login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [22]:
# Load all PDFs from the data folder
pdf_paths = glob.glob("./data/*.pdf")
all_docs = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    all_docs.extend(docs)

print(f"✅ Loaded {len(pdf_paths)} PDFs -> {len(all_docs)} pages total")


Multiple definitions in dictionary at byte 0x15287 for key /Ascent
Multiple definitions in dictionary at byte 0x244a7 for key /Ascent
Multiple definitions in dictionary at byte 0x300d8 for key /Ascent
Multiple definitions in dictionary at byte 0x3b14f for key /Ascent


✅ Loaded 5 PDFs -> 97 pages total


In [23]:
# Finer-grained chunking setup
CHUNK_SIZE = 400
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", "! ", "? ", " ", ""]
)

splits = text_splitter.split_documents(all_docs)
print(f"✅ Split into {len(splits)} chunks using chunk_size={CHUNK_SIZE}, overlap={CHUNK_OVERLAP}")

# Preview first few chunks
for i, chunk in enumerate(splits[:3]):
    print(f"\n--- Chunk {i+1} ---\n{chunk.page_content[:300]}")


✅ Split into 874 chunks using chunk_size=400, overlap=50

--- Chunk 1 ---
Deep Learning for Financial Applications : A Survey
Ahmet Murat Ozbayoglua, Mehmet Ugur Gudeleka, Omer Berat Sezera
aDepartment of Computer Engineering, TOBB University of Economics and Technology, Ankara, Turkey
Abstract
Computational intelligence in ﬁnance has been a very popular topic for bothaca

--- Chunk 2 ---
in various models. Meanwhile, within the Machine Learning (ML) ﬁeld, Deep Learning
(DL) started getting a lot of attention recently, mostly due to its outperformance over the
classical models. Lots of diﬀerent implementations of DL exist today, and the broad interest
is continuing. Finance is one pa

--- Chunk 3 ---
however, the playﬁeld is wide open, a lot of research opportunities still exist. In this paper,
we tried to provide a state-of-the-art snapshot of the developed DL models for ﬁnancial
applications, as of today. We not only categorized the works according to their intended
subﬁeld in ﬁnance

In [24]:
# Use free SentenceTransformer embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS index
vectorstore = FAISS.from_documents(splits, embeddings)
print("✅ FAISS vectorstore built successfully")


✅ FAISS vectorstore built successfully


In [56]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline

# Load model locally
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    tokenizer="google/flan-t5-base",
    max_new_tokens=512
)

# Wrap it for LangChain
llm = HuggingFacePipeline(pipeline=generator)

print("✅ Local Hugging Face FLAN-T5 model loaded successfully!")


config.json: 0.00B [00:00, ?B/s]

C:\Users\Aniketh\Desktop\RAG_Workshop\rag_env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aniketh\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regu

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ Local Hugging Face FLAN-T5 model loaded successfully!


In [62]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 6, "lambda_mult": 0.3}
)
print("✅ Retriever configured with tuned parameters")


✅ Retriever configured with tuned parameters


In [63]:
# simple summarizer using the same LLM (but with a short prompt)
summ_template = "Summarize the following context into 60-120 words:\n\n{context}\n\nSummary:"
# apply summarizer to retrieved docs (batch) then use summary as context

In [64]:
# Create the RAG pipeline (retriever + prompt + LLM)
template = """You are a helpful assistant that answers based on the provided context.

Context:
{context}

Question:
{question}

Answer concisely and accurately.
"""

prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG chain ready for testing")


✅ RAG chain ready for testing


In [58]:
questions = [
    "What are the main differences between LSTM and ARIMA models for stock price forecasting?",
    "List three practical risks of using ML models for algorithmic trading mentioned by regulators."
]

for q in questions:
    print("\n" + "="*80)
    print(f"🔍 Question: {q}")
    print("-"*80)
    answer = rag_chain.invoke(q)
    print(answer)



🔍 Question: What are the main differences between LSTM and ARIMA models for stock price forecasting?
--------------------------------------------------------------------------------


Token indices sequence length is longer than the specified maximum sequence length for this model (1789 > 512). Running this sequence through the model will result in indexing errors


LSTM neural network models are effective in predicting nthe stock market, and compared with other machine nlearning algorithms, LSTM neural network models nperform very well when applied to longer prediction nhorizons.

🔍 Question: List three practical risks of using ML models for algorithmic trading mentioned by regulators.
--------------------------------------------------------------------------------
There might be substantial risks of reinforcement learning based trading
